In [5]:
# focus on identfying director matches.

import pandas
import pathlib

acmi_creators = pandas.read_csv(pathlib.Path.cwd().parents[0] / 'acmi-api' / 'app' / 'tsv' / 'works.tsv', delimiter='\t', low_memory=False)


# acmi_creators = acmi_creators.dropna(subset='date_of_death')
# acmi_creators = acmi_creators.loc{}
# acmi_creators = acmi_creators.astype(str)
# acmi_creators = acmi_creators[['id', 'name']].rename(columns={'id':'creator_id'})


acmi_creators = acmi_creators[['creator_credit', 'credit_line']].dropna()
print(acmi_creators.columns.values)

print(len(acmi_creators))
acmi_creators.head()

['creator_credit' 'credit_line']
554


,creator_credit,credit_line
44,Mitchell Camera Corporation,<p>ACMI Collection</p>
178,Maker not documented,"<p>Issue 7, January 1996</p>"
183,Pathé Frères,<p>ACMI Collection. Donated by Martin Hudson</p>
211,National Panasonic and Matsushita Electrical C...,<p>ACMI Collection. Courtesy Andrew Evans.</p>
212,Apple Computer Inc.,<p>ACMI Collection</p>


In [11]:

import json
import numpy
import pandas
import pathlib
import pydash
import requests
import tqdm

acmi_path = pathlib.Path.cwd().parents[0] / 'acmi-api' / 'app' / 'json' / 'works'
acmi_files = [filename for filename in acmi_path.iterdir() if filename.suffix == '.json' and 'index' not in filename.name]

directors = list()

for acmi_file in tqdm.tqdm(acmi_files):

    with open(acmi_file, encoding='utf-8') as acmi_data:
        acmi_data = json.load(acmi_data)

    if 'creators_primary' in acmi_data:
        for y in acmi_data['creators_primary']:
            if 'role' in y:
                if y['role'] == 'director':
                    directors.append(y['creator_id'])

print(len(pydash.uniq(directors)))

100%|██████████| 42920/42920 [01:05<00:00, 651.41it/s] 


7548
